In [32]:
import os, subprocess, re

In [33]:
def runNSave(cmd, path, get_times=True):
    # result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.rstrip()
    # print("out:", out)
    # print("error:", error)
        
    with open(path, 'w') as fh:
        fh.write(out)
    
    if get_times:
        netw_time = int(re.search("networking \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        reas_time = int(re.search("reasoning \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        return netw_time, reas_time

def record(times_file, query, data, type, phase, netw_time, reas_time):
    times_file.write(f"{query},{data},{type},{phase},{netw_time},{reas_time}\n")

## Convert SPARQL PP into N3 PP

In [34]:
# NOTE for now run with Python 3.11.5 (manually fixed issue with NegatedPropertySet & reverse paths)

from rdflib.plugins.sparql import parser
from convert_pp_sparql_n3 import To_N3_Visitor

In [ ]:
# rdflib.plugins.sparql.parserutils.py#279
# add:
# elif isinstance(t,CompValue) or isinstance(t,URIRef):
#     res['part'] = t

### Convert

In [35]:
# convert single query

query = "SELECT ?x WHERE { ?x (:p1/:p2)* ?z ; !(:p3|:p4|:p5) ?a }"
query = "PREFIX : <http://example.org/gmark/> " + \
    "SELECT * WHERE { ?x0 !(^:p1|:p2) ?x3 } "
    # "SELECT * WHERE { ?x0 ((^:p1/:p2*)?/:p3)+ ?x3 } "
    # "SELECT * WHERE { ?x0 !(:p1|^:p2|:p3) ?x3 }"

query = parser.parseQuery(query)
query = query[1]

print(To_N3_Visitor().convert(query))

{ ?x0 ( '!' (( '^' :p1) '|' :p2)) ?x3 }
 => { :x :result true } .


In [36]:
# convert query folder

import os

visitor = To_N3_Visitor()

path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    print(file)
    with open(os.path.join(path, file), 'r') as fh:
        query = fh.read()
        query = parser.parseQuery(query)
        query = query[1]
        
        conv = visitor.convert(query)
        conv = "@prefix : <http://example.org/gmark/> .\n\n" + conv
        # print(conv)
        
        n3_file = file[0:file.index(".")] + ".n3"
        with open(os.path.join(path, "n3", n3_file), 'w') as fh2:
            fh2.write(conv)
            
        # print("\n")

query-0.sparql
query-1.sparql
query-10.sparql
query-11.sparql
query-12.sparql
query-13.sparql
query-14.sparql
query-15.sparql
query-16.sparql
query-17.sparql
query-18.sparql
query-19.sparql
query-2.sparql
query-20.sparql
query-21.sparql
query-22.sparql
query-23.sparql
query-24.sparql
query-25.sparql
query-26.sparql
query-27.sparql
query-28.sparql
query-29.sparql
query-3.sparql
query-30.sparql
query-31.sparql
query-32.sparql
query-33.sparql
query-34.sparql
query-35.sparql
query-36.sparql
query-37.sparql
query-38.sparql
query-39.sparql
query-4.sparql
query-40.sparql
query-41.sparql
query-42.sparql
query-43.sparql
query-44.sparql
query-45.sparql
query-46.sparql
query-47.sparql
query-48.sparql
query-49.sparql
query-5.sparql
query-6.sparql
query-7.sparql
query-8.sparql
query-9.sparql


### Ground & normalize

In [37]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"
for file in os.listdir(path):    
    if not file.endswith(".sparql"):
        continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, "n3", file)
    norm_file = os.path.join(path, "n3", "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times=False)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times=False)

query-24.n3
query-47.n3
query-36.n3
query-28.n3
query-12.n3
query-1.n3
query-34.n3
query-49.n3
query-38.n3
query-45.n3
query-26.n3
query-3.n3
query-10.n3
query-18.n3
query-7.n3
query-14.n3
query-41.n3
query-22.n3
query-30.n3
query-16.n3
query-5.n3
query-9.n3
query-32.n3
query-20.n3
query-43.n3
query-0.n3
query-13.n3
query-46.n3
query-25.n3
query-29.n3
query-37.n3
query-11.n3
query-2.n3
query-48.n3
query-35.n3
query-27.n3
query-44.n3
query-39.n3
query-23.n3
query-40.n3
query-31.n3
query-19.n3
query-15.n3
query-6.n3
query-33.n3
query-42.n3
query-21.n3
query-4.n3
query-17.n3
query-8.n3


## Run SPARQL PP

In [38]:
from resCSV2N3 import convert as csv2n3
# from rdflib import Graph

In [39]:
def exec_sparql(query_file, data_file, result_file):
    process = subprocess.Popen(['java', '-jar', "../test/run/sparql.jar", "-n3", data_file, "-query", query_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    # print(out); print(error)
    
    # g = Graph(); g.parse(data_file)
    # g.query(open(query_file, 'r').read())
    
    with open(result_file, 'w') as fh:
        fh.write(out.strip())

In [40]:
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    # print(file)
    
    name = file[:file.index(".")]
    query_file = os.path.join(path, file)
    data_file = os.path.join(path, "data.n3")
    result_file_csv = os.path.join(path, "results", f"{name}.csv")
    result_file_n3 = os.path.join(path, "results", f"{name}.n3")
    
    exec_sparql(query_file, data_file, result_file_csv)
    
    csv2n3(file=result_file_csv , ordered=False, out=result_file_n3)
        
    break

## Run N3 PP

In [41]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"

times_file = open(os.path.join(path, "n3", "results", "times.csv"), 'w')
times_file.write("query,data,type,phase,netw_time,reas_time\n")

files = list(os.listdir(os.path.join(path, "n3", "normalized")))
files.sort()
for file in files:
    if not file.startswith("query") and not file.endswith(".n3"):
        continue
    
    name = file[:file.index(".")]
    norm_file = os.path.join(path, "n3", "normalized", file)
    data_file = "data.n3"
    data_path = os.path.join(path, data_file)
    dir_res_file = os.path.join(path, "n3", "results", "direct", file)
    bwd_res_file = os.path.join(path, "n3", "results", "bwd", file)
    fwd_res_file = os.path.join(path, "n3", "results", "fwd", file)
    
    print(norm_file)
    
    # - direct 
    netw_time, reas_time = runNSave(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file)
    record(times_file, file, data_file, 'direct', 'n/a', netw_time, reas_time)
    
    # - backward
    tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd.n3")
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'bwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file)
    record(times_file, file, data_file, 'bwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'bwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
        
    # - forward
    tmp_file = os.path.join(path, "n3", "gen", f"{name}_fwd.n3")
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'fwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file)
    record(times_file, file, data_file, 'fwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'fwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
    
    times_file.flush()

/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/n3/normalized/query-0.n3


KeyboardInterrupt: 